In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import squidpy as sq
from skimage import io
import scanpy as sc
import time
import os
from dotenv import load_dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from napari_spongepy import functions as fc

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [4]:
# Path prefix for the data folder, use variable from .env file or default
DATA_PATH_PREFIX = os.getenv("DATA_PATH_PREFIX", default="../data")
SMALL_IMAGE = os.getenv("SMALL_IMAGE", default="False") in ["True", "true", "t"]
DEVICE = os.getenv("DEVICE", default="cuda:4")

name_slide = "A1-1"
path_coordinates = DATA_PATH_PREFIX + "/resolve_liver/20272_slide1_" + name_slide + "_results.txt"
path_image = DATA_PATH_PREFIX + "/resolve_liver/20272_slide1_" + name_slide + "_DAPI.tiff"

img = io.imread(path_image)
ic = sq.im.ImageContainer(img)

In [5]:
ic_correct, flatfield = fc.tilingCorrection(img=ic, left_corner=[2144, 2144] if SMALL_IMAGE else None, size=[2144, 2144] if SMALL_IMAGE else None)
fc.tilingCorrectionPlot(ic_correct.data.image.squeeze().to_numpy(), flatfield, img)

: 

: 

In [ ]:
crd = [3000, 6000, 5000, 9000] # a small subset of interest for visualization purposes.

if SMALL_IMAGE:
    # Smaller crd for demo purposes
    crd = [2000, 4000, 2000, 4000]
t0 = time.time()
ic_preproces = fc.preprocessImage(img=ic_correct, size_tophat=45, contrast_clip=3.5)
print(time.time() - t0)
fc.preprocessImagePlot(ic_preproces.data.image.squeeze().to_numpy(), ic_correct.data.image.squeeze().to_numpy(), small_size_vis=crd)

In [ ]:
img_preproces = ic_preproces.data.image.squeeze().to_numpy()
t0 = time.time()
masks, mask_i, polygons = fc.segmentation(img_preproces, device=DEVICE, cellprob_threshold=-4, flow_threshold=0.9, diameter=50, model_type="nuclei")
print(time.time() - t0)
fc.segmentationPlot(img_preproces, mask_i, polygons, small_size_vis=crd)

In [ ]:
adata = fc.create_adata_quick(path_coordinates, ic_preproces, masks)
fc.plot_shapes(adata)

In [ ]:
adata, adata_orig = fc.preprocessAdata(adata, masks)
fc.preprocesAdataPlot(adata, adata_orig)

In [ ]:
adata, filtered = fc.filter_on_size(adata, min_size=500)
print(str(filtered) + " cells were filtered out based on size.")

In [ ]:
adata = fc.extract(sq.ImageContainer(img), adata)
adata.obsm["segmentation_features"].head()

# plot segmentation features
sc.pl.spatial(
    sq.pl.extract(adata, "segmentation_features"),
    color=[
        "segmentation_label",
        "segmentation_area_mean",
    ],
    bw=True,
    ncols=2,
    vmin="p1",
    vmax="p99",
)